In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from pyvirtualdisplay import Display
import csv
import pandas as pd
import os
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

START_PAGE = 20
END_PAGE = 22
PROJECT_ABBR = "QS"
PROJECT_NAME = "Project 5"
URL = "https://www.topuniversities.com/university-rankings/world-university-rankings/2018"

driver =webdriver.Firefox()
driver.get(URL)

"""display = Display(visible=0, size=(800, 600))
display.start()"""

df_columns = ['Ranking',
              'University Name',
              'Country',
              'Source',
              'No. of Students',
              'Public/Private University',
              'Focus']

df_etf = pd.DataFrame(columns=df_columns)

def go_to_page(page):
    if page != 1:
        for i in range(page-1):
            next_page()
    else:
        return

def set_result_per_page():

    """select = Select(driver.find_element_by_xpath("//select[@name='qs-rankings_length']"))
    select.select_by_visible_text("200")
    for option in select.find_elements_by_tag_name('option'):
        if option.text == '200':
            option.click() # select() in earlier versions of webdriver
            break"""
    driver.find_element_by_xpath("//select[@name='qs-rankings_length']").send_keys('100')
    time.sleep(3)
    
def next_page():
    #click next page
    driver.find_element_by_xpath('//*[@id="qs-rankings_next"]').click()
    time.sleep(2)
    
def dig(link, row_index):
    #open a new window
    driver2 = webdriver.Firefox()
    driver2.get(link.get_attribute('href'))
    time.sleep(2)
    
    #find fields on the new driver
    df_etf.loc[row_index, 'No. of Students'] = driver2.find_element_by_xpath('//*[@id="uni-info"]/li[1]/h4/span').text
    df_etf.loc[row_index, 'Public/Private University'] = driver2.find_element_by_xpath('//*[@id="uni-info"]/li[3]/div/h4/span').text
    df_etf.loc[row_index, 'Focus'] = driver2.find_element_by_xpath('//*[@id="uni-info"]/li[5]/div/h4/span').text
    
    #close the driver
    driver2.quit()

def main():
    #set_result_per_page()
    
    go_to_page(START_PAGE)
    time.sleep(3)
    print 'Start at PAGE'+ str(START_PAGE)
    
    n = 1
    for CURRENT_PAGE in range(START_PAGE,END_PAGE+1):
        print '--------------------PAGE ' + str(CURRENT_PAGE) +' START--------------------'
        START_TIMESTAMP = time.time()
        for row in driver.find_elements_by_xpath('//*[@id="qs-rankings"]/tbody/tr'):
            df_etf.loc[n, 'Ranking']         = row.find_element_by_xpath('td[1]/div/div/span[2]').text
            df_etf.loc[n, 'University Name'] = row.find_element_by_xpath('td[2]/div/a').text
            df_etf.loc[n, 'Country']         = row.find_element_by_xpath('td[3]/div/img').get_attribute('alt')
            
            #get the link for the university page
            link = row.find_element_by_xpath('td[2]/div/a')
            #open the page to get other fields
            dig(link, n)
            n = n + 1
        print '--------------------PAGE ' + str(CURRENT_PAGE) +' FINISH--------------------'
        END_TIMESTAMP = time.time()
        print '-TIME USED:' + str(START_TIMESTAMP-END_TIMESTAMP) + '-'
        
        #store every first 5,10,15... pages to CSV for backup purpose
        if (CURRENT_PAGE % 1) == 0:
            filename = '['+ PROJECT_ABBR +'][Intrim Backup][' + str(CURRENT_PAGE) + ']' + PROJECT_NAME + ' PAGE ' + str(START_PAGE) + ' to PAGE ' + str(CURRENT_PAGE) + ' Records.csv'
            df_etf.to_csv(filename, index=False,  encoding='utf-8')
            print '[Intrim Backup]Saved First '+ str(CURRENT_PAGE) + 'Pages Records to CSV'
        
        #turn to next page
        next_page()
    
    #set Source as QS for every row as this is crawl from QS
    df_etf['Source'] = PROJECT_ABBR
    
    #export the file to CSV
    filename = '['+ PROJECT_ABBR +']'+ PROJECT_NAME +' PAGE ' + str(START_PAGE) + 'to PAGE ' + str(END_PAGE) + 'Records.csv'
    df_etf.to_csv(filename, index=False, encoding='utf-8')
    print 'Exported to csv'
    
    #close the driver
    driver.quit()
if __name__ == "__main__":
    main()



Start at PAGE20
--------------------PAGE 20 START--------------------


WebDriverException: Message: Failed to decode response from marionette


In [15]:
df_etf['Source'] = 'aaa'


In [20]:
df_etf

,Ranking,University Name,Country,Source,No. of Students,Public/Private University,Focus
1,1,Massachusetts Institute of Technology (MIT),United States,QS,"<12,000",Private,
2,2,Stanford University,United States,QS,,,
3,3,Harvard University,United States,QS,,,
4,4,California Institute of Technology (Caltech),United States,QS,,,
5,5,University of Cambridge,United Kingdom,QS,,,
6,6,University of Oxford,United Kingdom,QS,,,
7,7,UCL (University College London),United Kingdom,QS,,,
8,8,Imperial College London,United Kingdom,QS,,,
9,9,University of Chicago,United States,QS,,,
10,10,ETH Zurich - Swiss Federal Institute of Techno...,Switzerland,QS,,,
